## Import Libraries 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.layers import MaxPooling2D, Dropout, Flatten, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
# -1 is to hide gpu device. Remove this value to use Tensorflow GPU if you have CUDA installed.
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
# Define epochs and batch size
epochs = 20
bs = 32

## Preparing train and test set 

#### ImageDataGenerator is used for data augmentation. Data augmentation is a technique to artificially create new training data from existing training data.
#### This will improve accuracy and add more training data to the model.

In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [5]:
training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (224, 224),
                                                 batch_size = bs,
                                                 class_mode = 'categorical')

Found 3392 images belonging to 2 classes.


In [6]:
test_datagen = ImageDataGenerator(rescale = 1./255)

In [7]:
test_set = test_datagen.flow_from_directory('dataset/test',
                                            target_size = (224, 224),
                                            batch_size = bs,
                                            class_mode = 'categorical')

Found 441 images belonging to 2 classes.


## Load and build the model 

#### MobileNetV2 is a lightweight Keras application with less no of parameters and depth.  It takes an input size of (224, 224, 3)

In [8]:
IMAGE_SIZE = [224, 224]
mobilenet = MobileNetV2(input_shape = IMAGE_SIZE + [3], weights = 'imagenet', include_top = False )

In [9]:
# We want to use the imagenet weights and add few layers on top of the MobileNetV2.
for layer in mobilenet.layers:
  layer.trainable = False

In [10]:
headModel = mobilenet.output
headModel = MaxPooling2D(pool_size = (7, 7))(headModel)
headModel = Flatten()(headModel)
headModel = Dense(128, activation = "relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation = "softmax")(headModel)

In [11]:
model = Model(inputs = mobilenet.input, outputs = headModel)

In [12]:
opt = Adam(learning_rate = 1e-3, decay = 1e-3 // epochs)
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [13]:
# This callback will save the model which has the best accuracy in epoch, as 'MobileNetV2.h5'
checkpoint = ModelCheckpoint('MobileNetV2.h5', monitor = 'accuracy', save_best_only = True)

In [14]:
model.fit(
  training_set,
  validation_data = test_set,
  epochs = epochs,
  steps_per_epoch = len(training_set) // bs,
  validation_steps = len(test_set) // bs,
  workers = 4,
  callbacks = checkpoint
)

Epoch 1/20
3/3 [==============================] - 7s 1s/step - loss: 2.8577 - accuracy: 0.6146
Epoch 2/20
3/3 [==============================] - 4s 1s/step - loss: 1.8757 - accuracy: 0.6354
Epoch 3/20
3/3 [==============================] - 4s 964ms/step - loss: 0.5544 - accuracy: 0.8646
Epoch 4/20
3/3 [==============================] - 3s 935ms/step - loss: 0.6765 - accuracy: 0.8333
Epoch 5/20
3/3 [==============================] - 4s 924ms/step - loss: 0.4635 - accuracy: 0.8333
Epoch 6/20
3/3 [==============================] - 4s 941ms/step - loss: 0.4340 - accuracy: 0.8854
Epoch 7/20
3/3 [==============================] - 3s 913ms/step - loss: 0.4053 - accuracy: 0.9062
Epoch 8/20
3/3 [==============================] - 3s 908ms/step - loss: 0.2171 - accuracy: 0.9167
Epoch 9/20
3/3 [==============================] - 3s 942ms/step - loss: 0.2272 - accuracy: 0.9167
Epoch 10/20
3/3 [==============================] - 4s 949ms/step - loss: 0.1406 - accuracy: 0.9479
Epoch 11/20
3/3 [========

## Now Let's test the model with a image from test set

In [15]:
from tensorflow.keras.preprocessing import image
test_image = image.load_img('dataset/test/with_mask/253.jpg', target_size = (224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = preprocess_input(test_image)
result = model.predict(test_image)

In [16]:
training_set.class_indices

{'with_mask': 0, 'without_mask': 1}

In [17]:
if np.argmax(result[0]) == 0:
  print('With Mask')
else:
  print('Without Mask')

With Mask
